# Text Preprocessing
:label:`sec_text_preprocessing`

We have reviewed and evaluated
statistical tools 
and prediction challenges
for sequence data.
Such data can take many forms.
Specifically,
as we will focus on
in many chapters of the book,
text is one of the most popular examples of sequence data.
For example,
an article can be simply viewed as a sequence of words, or even a sequence of characters.
To facilitate our future experiments
with sequence data,
we will dedicate this section
to explain common preprocessing steps for text.
Usually, these steps are:

1. Load text as strings into memory.
1. Split strings into tokens (e.g., words and characters).
1. Build a table of vocabulary to map the split tokens to numerical indices.
1. Convert text into sequences of numerical indices so they can be manipulated by models easily.


In [1]:
%use @file[../djl.json]
// %load ../utils/djl-imports

## Reading the Dataset

To get started we load text from H. G. Wells' [*The Time Machine*](http://www.gutenberg.org/ebooks/35).
This is a fairly small corpus of just over 30000 words, but for the purpose of what we want to illustrate this is just fine.
More realistic document collections contain many billions of words.
The following function reads the dataset into a list of text lines, where each line is a string.
For simplicity, here we ignore punctuation and capitalization.


In [2]:
import java.net.URL
import java.io.BufferedReader
import java.io.InputStreamReader
import java.util.Locale
fun readTimeMachine(): Array<String> {
    val url = URL("http://d2l-data.s3-accelerate.amazonaws.com/timemachine.txt")
    var lines: Array<String>
    BufferedReader(InputStreamReader(url.openStream())).use { inp ->
        lines = inp.readLines().toTypedArray()
    }
    for (i in lines.indices) {
        lines[i] = lines[i].replace("[^A-Za-z]+".toRegex(), " ").trim().lowercase(Locale.getDefault())
    }
    return lines
}

val lines = readTimeMachine()
println("# text lines: " + lines.size);
println(lines[0])
println(lines[10])

# text lines: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## Tokenization

The following `tokenize` function
takes an array (`lines`) as the input,
where each element is a text sequence (e.g., a text line).
Each text sequence is split into a list of tokens.
A *token* is the basic unit in text.
In the end,
a list of token lists are returned,
where each token is a string.


In [3]:
fun tokenize(lines: Array<String>, token: String) : List<List<String>> {
    // Split text lines into word or character tokens.
//    String[][] output = new String[lines.length][];
    val output = mutableListOf<List<String>>()
    if (token == "word") {
        for (i in 0 until lines.size) {
            output.add(lines[i].split(" "))
        }
    } else if (token == "char") {
        for (i in 0 until lines.size) {
            output.add(lines[i].split(""))
        }
    } else {
        throw Exception("ERROR: unknown token type: " + token);
    }
    return output; 
}

val tokens = tokenize(lines, "word");
for (i in 0 until 11) {
    println(tokens[i]);
}

[the, time, machine, by, h, g, wells]
[]
[]
[]
[]
[i]
[]
[]
[the, time, traveller, for, so, it, will, be, convenient, to, speak, of, him]
[was, expounding, a, recondite, matter, to, us, his, grey, eyes, shone, and]
[twinkled, and, his, usually, pale, face, was, flushed, and, animated, the]


## Vocabulary

The string type of the token is inconvenient to be used by models, which take numerical inputs.
Now let us build a dictionary (HashMap), often called *vocabulary* as well, to map string tokens into numerical indices starting from 0.
To do so, we first count the unique tokens in all the documents from the training set,
namely a *corpus*,
and then assign a numerical index to each unique token according to its frequency.
Rarely appeared tokens are often removed to reduce the complexity.
Any token that does not exist in the corpus or has been removed is mapped into a special unknown token “&lt;unk&gt;”.
We optionally add a list of reserved tokens, such as
“&lt;pad&gt;” for padding,
“&lt;bos&gt;” to present the beginning for a sequence, and “&lt;eos&gt;” for the end of a sequence.


In [15]:
class Vocab(tokens: List<List<String>>, minFreq: Int, reservedTokens: List<String>) {
    // The index for the unknown token is 0
    var unk: Int= 0
    var tokenFreqs: kotlin.collections.List<kotlin.Pair<String, Int>>
    var idxToToken: MutableList<String> = mutableListOf()
    var tokenToIdx: MutableMap<String, Int> = mutableMapOf()

    init {
        // Sort according to frequencies
        val counter = countCorpus2D(tokens)
        tokenFreqs = counter.toList().sortedBy { (_, value) -> value}

        val uniqTokens: MutableSet<String> = mutableSetOf()
        uniqTokens.add("<unk>")
        uniqTokens.addAll(reservedTokens)
        uniqTokens.addAll(tokenFreqs.filter{(key, value) -> 
            value >= minFreq && !uniqTokens.contains(key)
        }.map { it.first })
        for (token in uniqTokens) {
            idxToToken.add(token)
            tokenToIdx[token] = idxToToken.size - 1
        }
    }

    fun length(): Int {
        return idxToToken.size
    }

    fun getIdxs(tokens: List<String>): List<Int> {
        return tokens.map { getIdx(it) }
    }

    fun getIdx(token: String): Int {
        return tokenToIdx.getOrDefault(token, unk)
    }
}

    fun countCorpus(tokens: List<String>): Map<String, Int> {
        /* Count token frequencies. */
        var counter = tokens.groupingBy { it }.eachCount()
        return counter
    }

    fun countCorpus2D(tokens: List<List<String>>): Map<String, Int> {
        /* Flatten a list of token lists into a list of tokens */
        return countCorpus(tokens.flatten().filter{ it != ""})
    }

We construct a vocabulary using the time machine dataset as the corpus. 
Then we print the first few frequent tokens with their indices.

In [16]:
//println(tokens)
val vocab = Vocab(tokens, 0, listOf<String>());
for (i in 0 until 10) {
    val token = vocab.idxToToken.get(i)
    print("(" + token + ", " + vocab.tokenToIdx.get(token) + ") ")
}

(<unk>, 0) (h, 1) (g, 2) (recondite, 3) (twinkled, 4) (radiance, 5) (incandescent, 6) (lights, 7) (lilies, 8) (bubbles, 9) 

Now we can convert each text line into a list of numerical indices.


In [17]:
for (i in intArrayOf(0,10)) {
    println("Words:" + tokens[i])
    println("Indices:" + vocab.getIdxs(tokens[i]))
}

Words:[the, time, machine, by, h, g, wells]
Indices:[4579, 4561, 4528, 4540, 1, 2, 4142]
Words:[twinkled, and, his, usually, pale, face, was, flushed, and, animated, the]
Indices:[4, 4577, 4555, 3161, 4181, 4466, 4573, 2399, 4577, 3162, 4579]


## Putting All Things Together

Using the above functions, we package everything into the `loadCorpusTimeMachine` function, which returns `corpus`, a list of token indices, and `vocab`, the vocabulary of the time machine corpus.
The modifications we did here are:
i) we tokenize text into characters, not words, to simplify the training in later sections;
ii) `corpus` is a single list, not a list of token lists, since each text line in the time machine dataset is not necessarily a sentence or a paragraph.


In [18]:
fun  loadCorpusTimeMachine(maxTokens: Int) : kotlin.Pair<List<Int>, Vocab> {
    /* Return token indices and the vocabulary of the time machine dataset. */
    val lines = readTimeMachine()
    val tokens = tokenize(lines, "char");
    val vocab = Vocab(tokens, 0, listOf<String>());
    // Since each text line in the time machine dataset is not necessarily a
    // sentence or a paragraph, flatten all the text lines into a single list
    var corpus = tokens.flatten().filter{it !=""}.map{vocab.getIdx(it)}
    if (maxTokens > 0) {
        corpus = corpus.subList(0, maxTokens);
    }
    return kotlin.Pair(corpus, vocab);
}

val corpusVocabPair = loadCorpusTimeMachine(-1);
val corpus = corpusVocabPair.first
val vocab = corpusVocabPair.second

println(corpus.size)
println(vocab.length())

170580
28


## Summary

* Text is an important form of sequence data.
* To preprocess text, we usually split text into tokens, build a vocabulary to map token strings into numerical indices, and convert text data into token indices for  models to manipulate.


## Exercises

1. Tokenization is a key preprocessing step. It varies for different languages. Try to find another three commonly used methods to tokenize text.
1. In the experiment of this section, tokenize text into words and vary the `minFreq` arguments of the `Vocab` instance. How does this affect the vocabulary size?